# 

In [1]:
# Import libraries
import requests
import re # regular expressions
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import unidecode
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from time import sleep

In [2]:
# local variables
subCategorListTitle  = "j-menu-item"
CHROME_DRIVER_PATH = r'C:\Users\User\Downloads\chromedriver_win32\chromedriver.exe'
URL = 'https://www.wildberries.ru/catalog/muzhchinam/odezhda/bryuki-i-shorty'


driver = webdriver.Chrome(CHROME_DRIVER_PATH)
driver.get(URL)
driver.maximize_window()

content = driver.page_source
soup = BeautifulSoup(content)

prices = []
brands = []
categories = []

data = {
        "Categories" : categories,
        "Brand" : brands,
        "Price" : prices
}

df = pd.DataFrame(data)

In [3]:
#To wait till page loads
def waitTillPageLoads(classNameLoaded, URL_name):
    driver.get(URL_name)
    delay = 10
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, classNameLoaded)))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
        
waitTillPageLoads('lower-price', driver.current_url)


Page is ready!


In [4]:
def searchAllItemsOfCurrentPage(prices, brands, categories):
    class_title = 'j-card-item'
    res = []
    sizes_available = []
    content = driver.page_source
    soup = BeautifulSoup(content)
    for block in soup.findAll('div', attrs={'class':class_title}):
        class_title = 'j-card-item'

        # Get price of clothes
        priceString = "0"
        priceBlock = block.find('ins', attrs={'class': 'lower-price'})
        if(priceBlock != None):
            priceString = block.find('ins', attrs={'class': 'lower-price'}).text
        elif (block.find('span', attrs={'class': 'lower-price'}) == None):
            priceString = "0"
        else:
            priceString = block.find('span', attrs={'class': 'lower-price'}).text
        price = unidecode.unidecode(priceString.strip())
        prices.append(price)
        # Get brand of clothes
        brandString = block.find('strong', attrs={'class': 'brand-name'}).text
        brand = unidecode.unidecode(brandString.strip())
        brands.append(brand)

        # Get category of clothes
        categoryString = block.find('span', attrs={'class': 'goods-name'}).text
        category = unidecode.unidecode(categoryString.strip())
        categories.append(category)
        sizes = []
    res.append(categories)
    res.append(brands)
    res.append(prices)
    addDataToDF(res, df)
        # todo
        # Get sizes of clothes
        
    #    for sizes_block in block.find('span', attrs = {'class': 'product-card__sizes'}):
    #        sizeString = sizes_block.find('a', attrs={'class': 'j-size'}).text
    #        size = unidecode.unidecode(sizeString.strip())
    #        sizes.append(size)
    #        print(sizes)
    #     sizes_available.append(sizes)

In [5]:
def addDataToDF(list, df):
    count = 0
    df2 = pd.DataFrame(data=list).T
    df2.columns = ["Categories", "Brand", "Price"]
    df = pd.concat([df, df2])
    df.to_excel('data.xlsx',index=False)
    print(df)

In [6]:
#buttons
burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")



prices = []
brands = []
categories = []



#Search for shoes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    shoesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--629")
    ActionChains(driver).click(burgerButton).pause(1).click(shoesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 5:
        break

        
        
        
        
#Search for women clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forWomenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--306")
    ActionChains(driver).click(burgerButton).pause(1).click(forWomenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 28:
        break
 

#Search for babies clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forBabiesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--115")
    ActionChains(driver).click(burgerButton).pause(1).click(forBabiesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 2:
        break
 
#Search for men clothes
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    forMenButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--566")
    ActionChains(driver).click(burgerButton).pause(1).click(forMenButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 26:
        break

#Search for accessories
count = 0
while True:
    burgerButton = driver.find_element(By.CLASS_NAME, "nav-element__burger")
    accessoriesButton = driver.find_element(By.CLASS_NAME, "menu-burger__main-list-link--1")
    ActionChains(driver).click(burgerButton).pause(1).click(accessoriesButton).perform()
    waitTillPageLoads("maincatalog-list-2", driver.current_url)
    catalogList = driver.find_element(By.CLASS_NAME, "maincatalog-list-2")
    items = catalogList.find_elements_by_tag_name("li")
    ActionChains(driver).click(items[count]).pause(1).perform()
    searchAllItemsOfCurrentPage(prices, brands, categories)
    count+=1
    
    if count == 18:
        break



Page is ready!
Empty DataFrame
Columns: [Categories, Brand, Price]
Index: []
Page is ready!
                                         Categories       Brand    Price
0                                            Sapogi       Reima  3 149 R
1   Zimnie botinki dlia mal'chika membrana, detskie       V-lux  8 855 R
2                                            Sapogi       Reima  3 149 R
3                                           Botinki       Reima  5 249 R
4                                            Sapogi       Reima  3 149 R
..                                              ...         ...      ...
95                                         Sandalii    Tereza&M  1 350 R
96                                         Sandalii    Tereza&M  1 350 R
97              Pinetki dlia malysha detskaia obuv'     Azaliia  1 152 R
98                      Pinetki dlia devochki obuv'    Tereza&M  1 050 R
99                                            Tufli  Kapitoshka    965 R

[100 rows x 3 columns]
Page is 

Page is ready!
                                          Categories        Brand     Price
0                                             Sapogi        Reima   3 149 R
1    Zimnie botinki dlia mal'chika membrana, detskie        V-lux   8 855 R
2                                             Sapogi        Reima   3 149 R
3                                            Botinki        Reima   5 249 R
4                                             Sapogi        Reima   3 149 R
..                                               ...          ...       ...
920              Kombinezon zhenskii zimnii flisovyi      KG-mart   2 028 R
921                                       Kombinezon  Moda Master  10 920 R
922                                       Kombinezon     TRESELLA   4 060 R
923                   Zhenskii kombinezon s zapakhom  Ashley Rose   1 850 R
924        Kombinezon shorty romper zhenskii chernyi  Ashley Rose   1 850 R

[925 rows x 3 columns]
Page is ready!
                                  

Page is ready!
                                           Categories                Brand  \
0                                              Sapogi                Reima   
1     Zimnie botinki dlia mal'chika membrana, detskie                V-lux   
2                                              Sapogi                Reima   
3                                             Botinki                Reima   
4                                              Sapogi                Reima   
...                                               ...                  ...   
1739    Khalat zhenskii domashnii makhrovyi na molnii          Flora Furia   
1740              Khalat zhenskii domashnii na molnii              Buy-tex   
1741            Khalat domashnii shelkovyi kruzhevnoi  Lovely lady's style   
1742             Zhenskii domashnii khalat "Nadezhda"         Matveev A.V.   
1743      Khalat zhenskii domashnii s zapakhom teplyi            Lan SARO.   

        Price  
0     3 149 R  
1     8 855 R  


Page is ready!
                                             Categories           Brand  \
0                                                Sapogi           Reima   
1       Zimnie botinki dlia mal'chika membrana, detskie           V-lux   
2                                                Sapogi           Reima   
3                                               Botinki           Reima   
4                                                Sapogi           Reima   
...                                                 ...             ...   
2465                                           Futbolka  Vittoria Vicci   
2466                                           Futbolka  Vittoria Vicci   
2467                                           Futbolka       VolgaTeks   
2468           Iubka trikotazhnaia mini oblegaiushchaia  Vittoria Vicci   
2469  Kostium zhenskii Sportivnyi tioplyi iz futera ...            TE0N   

        Price  
0     3 149 R  
1     8 855 R  
2     3 149 R  
3     5 249 R  
4   

Page is ready!
                                           Categories      Brand    Price
0                                              Sapogi      Reima  3 149 R
1     Zimnie botinki dlia mal'chika membrana, detskie      V-lux  8 855 R
2                                              Sapogi      Reima  3 149 R
3                                             Botinki      Reima  5 249 R
4                                              Sapogi      Reima  3 149 R
...                                               ...        ...      ...
3198                                           Kurtka      Gusti  6 900 R
3199                                       Kombinezon     Lassie  5 999 R
3200              Zimniaia kurtka iz membrannoi tkani  PlayToday  6 999 R
3201       Futbolka detskaia 2 sht belaia v sad fizra      t-sod    395 R
3202                                       Kombinezon     Lassie  5 999 R

[3203 rows x 3 columns]
Page is ready!
                                           Categories    

Page is ready!
                                           Categories          Brand    Price
0                                              Sapogi          Reima  3 149 R
1     Zimnie botinki dlia mal'chika membrana, detskie          V-lux  8 855 R
2                                              Sapogi          Reima  3 149 R
3                                             Botinki          Reima  5 249 R
4                                              Sapogi          Reima  3 149 R
...                                               ...            ...      ...
4017                                Maika sportivnaia  INFERNO style  1 661 R
4018                         Maika sportivnaia "Lion"  INFERNO style    584 R
4019                         Maika sportivnaia "Lion"  INFERNO style    918 R
4020                                Maika sportivnaia          zilla  1 440 R
4021                           Maika FTBR-TEAM RUSSIA       RFIGHTER  1 800 R

[4022 rows x 3 columns]
Page is ready!
         

Page is ready!
                                             Categories          Brand  \
0                                                Sapogi          Reima   
1       Zimnie botinki dlia mal'chika membrana, detskie          V-lux   
2                                                Sapogi          Reima   
3                                               Botinki          Reima   
4                                                Sapogi          Reima   
...                                                 ...            ...   
4940  Noski Noski muzhskie Noski muzhskie klassiches...    Premium LUX   
4941          Termobel'e muzhskoe dlia kholodnoi pogody           UXTi   
4942            Termobel'e muzhskoe sportivnoe komplekt         MUKAMA   
4943                                      Termokomplekt        NEL-dom   
4944                Teplyi muzhskoi termobel'e na flise  01 ICON.Brand   

        Price  
0     3 149 R  
1     8 855 R  
2     3 149 R  
3     5 249 R  
4     3 149 R  


Page is ready!
                                           Categories               Brand  \
0                                              Sapogi               Reima   
1     Zimnie botinki dlia mal'chika membrana, detskie               V-lux   
2                                              Sapogi               Reima   
3                                             Botinki               Reima   
4                                              Sapogi               Reima   
...                                               ...                 ...   
5761                    Tufli muzhskie Derbi kozhanye       Bruno Renzoni   
5762          Tufli muzhskie s dvumi priazhkami Monki  BOTTEGA PARTENOPEA   
5763                                            Tufli              Walrus   
5764                                            Tufli              Walrus   
5765              Tufli muzhskie kozhanye Polubotinki         Marco Rocco   

         Price  
0      3 149 R  
1      8 855 R  
2      3 

Page is ready!
                                             Categories              Brand  \
0                                                Sapogi              Reima   
1       Zimnie botinki dlia mal'chika membrana, detskie              V-lux   
2                                                Sapogi              Reima   
3                                               Botinki              Reima   
4                                                Sapogi              Reima   
...                                                 ...                ...   
6475              Zerkal'tse karmannoe v sumku skladnoe          SunSphynx   
6476                                         Zerkal'tse   Sergio Valentini   
6477  Zerkal'tse kosmeticheskoe skladnoe s 2Kh uveli...  MAGAZIN NA DIVANE   
6478                               Zerkal'tse karmannoe             RAISSA   
6479              Zerkal'tse karmannoe v sumku skladnoe          SunSphynx   

        Price  
0     3 149 R  
1     8 855 R  


Page is ready!
                                           Categories       Brand    Price
0                                              Sapogi       Reima  3 149 R
1     Zimnie botinki dlia mal'chika membrana, detskie       V-lux  8 855 R
2                                              Sapogi       Reima  3 149 R
3                                             Botinki       Reima  5 249 R
4                                              Sapogi       Reima  3 149 R
...                                               ...         ...      ...
7295        Kapor zhenskii zimnii viazanyi iz mokhera   Neushanka  1 341 R
7296    Nespadaiushchii platok dlia khrama v tserkov'  Platkov.RF    681 R
7297        Kapor zimnii angorskii iz angory zhenskii   Neushanka  1 512 R
7298       Kapor zhenskii viazanyi kapiushon na osen'   Neushanka  1 311 R
7299       Kapor zhenskii viazanyi kapiushon na osen'   Neushanka  1 040 R

[7300 rows x 3 columns]
Page is ready!
                                           Ca